In [ ]:
import random, noise, math

from collections import defaultdict
from heapq import *

## Node Class

Class to keep information used by the dijkstra algorithm. The move cost return how much it will cost to move from this node to another. Now it is calculated as the height difference between the two pluss a small constant to make sure it will take the shortest path along a flat plane.

In [ ]:
class Node:
    def __init__(self,value,point):
        self.value = value
        self.dist = 999999
        self.point = point
        self.parent = None

    def move_cost(self,other):
        if self.value > other.value:
            return self.value - other.value + 0.01
        else:
            return other.value - self.value + 0.01

## Dijkstra
### Helper Function
#### Get Neighbour

Returns all direct neighbours from a give point. It will not return diagonal neighbours.<br>
**Return**: A list of four nodes.

In [3]:
def get_neighbours(point, grid):
    x,y = point.point
    liste = []
    if x-1 > 0:
        liste.append((x-1, y))
    if x+1 < len(grid):
        liste.append((x+1, y))
    if y-1 > 0:
        liste.append((x, y-1))
    if y+1 < len(grid[0]):
        liste.append((x, y+1))

    return [grid[d[0]][d[1]] for d in liste]

#### Get Path

Traces a path back to the start using each nodes parents. This path will be the shortest assuming each nodes parrent is the one with the least travel distance.<br>
**Return**: List of nodes that is the most optimal path.

In [4]:
def get_path(current):
    path = []
    while current.parent:
        path.append(current)
        current = current.parent

    return path

#### Get Path Midpoint

Finds the median point in a path.<br>
**Return**: Single node that is the median point.

In [5]:
def get_path_midpoint(path):
    return path[math.floor(len(path) / 2)]

#### Update Neighbours

Loops through all neigbouring nodes to see if the travel distance through itself to the neighbour is shorter than its current distance. It will update all neighbours and add them to the openset if they have not been visited before, this is done to decrease the amount of node min() has to check.<br>
**Return**: Nothing

In [6]:
def update_neighbours(grid, current, visited, openset):
    for n in get_neighbours(current, grid):
        if (not visited.__contains__(n)):
            openset.add(n)
            if n.dist > current.dist + n.move_cost(current):
                n.dist = current.dist + n.move_cost(current)
                n.parent = current

#### Get Edges

Calculates movment costs between all nodes.<br>
**Return**: A list of movment cost between two points.

In [7]:
def get_edges(world):
    edges = []
    
    for r in world:
        for i in r:
            for n in get_neighbours(i, world):
                edges.append((i.point, n.point, i.move_cost(n)))
                
    return edges

#### Get Path

Gets all the points of a dijkstra path as a list of points.<br>
**Return**: A list of points.

In [8]:
def get_path(dijkstra_path):
    path = []
    c = dijkstra_path[1]
    while c:
        if (isinstance(c, Node) or isinstance(c, float)):
            break
        
        path.append(c[0])
        c = c[1]
        
    return path

#### Get Cost

Get the total cost of a dijkstra path.<br>
**Return**: float

In [9]:
def get_cost(dijkstra_path):
    return dijkstra_path[0]

### Dijkstra Algorithim Implementation

Dijkstra is the algorithm used for this data generation. This is an optimized version, for explonation see wikipedia.

In [10]:
def dijkstra(edges, f, t):
    g = defaultdict(list)
    for l,r,c in edges:
        g[l].append((c,r))

    q, seen, mins = [(0,f,())], set(), {f: 0}
    while q:
        (cost,v1,path) = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            path = (v1, path)
            if v1 == t: return (cost, path)

            for c, v2 in g.get(v1, ()):
                if v2 in seen: continue
                prev = mins.get(v2, None)
                next = cost + c
                if prev is None or next < prev:
                    mins[v2] = next
                    heappush(q, (next, v2, path))

    return float("inf")